In [6]:
import requests
from bs4 import BeautifulSoup
import hashlib
import os
import io
import zipfile

from azure.storage.blob import BlobServiceClient
from concurrent.futures import ThreadPoolExecutor

In [7]:
# URL of the webpage containing the .csv files
url = 'http://data.gdeltproject.org/gkg/index.html'  # Replace with the actual URL
base_url = 'http://data.gdeltproject.org/gkg/'

In [8]:
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
files = soup.find_all('a', href=True)
# # Example of parsing the webpage to find the links, sizes, and MD5s
# files = soup.find_all('a', href=True)
# gkg.csv.zip 

file_lists = []
for file in files:
    # print(file['href'])
    if file['href'].endswith('gkg.csv.zip'):
        file_lists.append(base_url+file['href'])

In [9]:
# Azure Storage account details
storage_account_name = "factoredatathon2024"
storage_account_key = "wgbe0Fzs4W3dPNc35dp//uumz+SPDXVLLGu0mNaxTs2VLHCCPnD7u79PYt4mKeSFboqMRnZ+s+ez+ASty+k+sQ=="
container_name = "gkg"

# Create a BlobServiceClient
blob_service_client = BlobServiceClient(
    f"https://{storage_account_name}.blob.core.windows.net", storage_account_key
)

# Function to download and upload a file
def download_and_upload(url):
    try:
        # Download the file
        file_name = os.path.basename(url)
        response = requests.get(url)
        local_path = f"/tmp/{file_name}"
        
        # Decompress the .zip file in memory
        with zipfile.ZipFile(io.BytesIO(response.content)) as z:
            for zip_info in z.infolist():
                # Extract the file to a byte stream
                with z.open(zip_info) as extracted_file:
                    # Upload the extracted file to Azure Data Lake Storage
                    blob_name = zip_info.filename  # Use the name of the file inside the .zip
                    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
                    blob_client.upload_blob(extracted_file.read(), overwrite=True)
                    print(f"Uploaded {blob_name} to Azure Data Lake Storage")

    except Exception as e:
        print(f"Failed to upload {file_name}: {e}")
        
# Use ThreadPoolExecutor to download and upload files concurrently
with ThreadPoolExecutor() as executor:
    executor.map(download_and_upload, file_lists)

Uploaded 20240811.gkg.csv to Azure Data Lake Storage
Uploaded 20240810.gkg.csv to Azure Data Lake Storage
Uploaded 20240809.gkg.csv to Azure Data Lake Storage
Uploaded 20240816.gkg.csv to Azure Data Lake Storage
Uploaded 20240813.gkg.csv to Azure Data Lake Storage
Uploaded 20240814.gkg.csv to Azure Data Lake Storage
Uploaded 20240812.gkg.csv to Azure Data Lake Storage
Uploaded 20240804.gkg.csv to Azure Data Lake Storage
Uploaded 20240815.gkg.csv to Azure Data Lake Storage
Uploaded 20240803.gkg.csv to Azure Data Lake Storage
Uploaded 20240807.gkg.csv to Azure Data Lake Storage
Uploaded 20240806.gkg.csv to Azure Data Lake Storage
Uploaded 20240805.gkg.csv to Azure Data Lake Storage
Uploaded 20240802.gkg.csv to Azure Data Lake Storage
Uploaded 20240808.gkg.csv to Azure Data Lake Storage
Uploaded 20240801.gkg.csv to Azure Data Lake Storage
Uploaded 20240728.gkg.csv to Azure Data Lake Storage
Uploaded 20240731.gkg.csv to Azure Data Lake Storage
Uploaded 20240726.gkg.csv to Azure Data Lake S